In [8]:
!uv pip install trl unsloth

Using Python 3.11.13 environment at: /usr
Resolved 83 packages in 1.00s
⠙ Preparing packages... (0/21)
⠙ Preparing packages... (0/21)
⠙ Preparing packages... (0/21)
unsloth-zoo ------------------------------ 14.91 KiB/160.84 KiB
⠙ Preparing packages... (0/21)
unsloth-zoo ------------------------------ 14.91 KiB/160.84 KiB
⠙ Preparing packages... (0/21)
unsloth-zoo ------------------------------ 14.91 KiB/160.84 KiB
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
⠙ Preparing packages... (0/21)
unsloth-zoo ------------------------------ 14.91 KiB/160.84 KiB
fsspec     ------------------------------     0 B/189.08 KiB
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
⠙ Preparing packages... (0/21)
unsloth-zoo ------------------------------ 14.91 KiB/160.84 KiB
fsspec     ------------------------------     0 B/189.08 KiB
nvidia-cuda-cupti-cu12 ------------------------------ 14.90 KiB/13.17 MiB
⠙ Preparing packages... (0/21)
unsloth-zoo --------

In [9]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True
 )

==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [11]:
#r is rank on low rank adaptation, r=16 is perfect balance between high and low ranks
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

Unsloth 2025.7.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [12]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [13]:
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [14]:
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [16]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [17]:
dataset=dataset.map(
    lambda samples:{
        "text":[tokenizer.apply_chat_template(convo,tokenize=False)
                for convo in samples["conversations"]
                ]

    },
      batched=True

)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [18]:
dataset

Dataset({
    features: ['conversations', 'source', 'score', 'text'],
    num_rows: 100000
})

In [19]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [20]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="outputs"
    ),
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [21]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vvishus (vvishus-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.439700
2,1.845800
3,1.380100
4,1.413600
5,1.350500
6,1.445600
7,0.946300
8,1.507000
9,1.239000
10,1.253800


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=60, training_loss=1.0262523502111436, metrics={'train_runtime': 549.4286, 'train_samples_per_second': 0.874, 'train_steps_per_second': 0.109, 'total_flos': 5495762487078912.0, 'train_loss': 1.0262523502111436})

In [22]:
model.save_pretrained("finetuned_model")

In [23]:
inference_model,inference_tokenizer=FastLanguageModel.from_pretrained(
    model_name="./finetuned_model",
    max_seq_length=2048,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [27]:
text_prompts=[
    "what are the key principles of Crypto investment?"
]

for prompt in text_prompts:
  formatted_prompt=inference_tokenizer.apply_chat_template(
      [{
        "role":"user",
        "content":prompt

      }],
      tokenize=False)

In [28]:
model_inputs=inference_tokenizer(formatted_prompt,return_tensors="pt").to("cuda")

In [29]:
generated_ids=inference_model.generate(
    **model_inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    pad_token_id=inference_tokenizer.pad_token_id

)

response=inference_tokenizer.batch_decode(generated_ids,skip_special_tokens=True)[0]
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 21 Jul 2025

user

what are the key principles of Crypto investment?assistant

1. **Understand the risks**: Cryptocurrencies are volatile and subject to market fluctuations. It's essential to understand the risks involved and to set a budget accordingly.
2. **Choose the right platform**: Select a reputable cryptocurrency exchange or platform that provides secure storage options, low fees, and reliable customer support.
3. **Research and due diligence**: Research the cryptocurrency you're interested in investing in, its underlying technology, and its potential use cases. Conduct thorough due diligence to ensure it's a viable investment opportunity.
4. **Diversify your portfolio**: Spread your investments across different cryptocurrencies and asset classes to minimize risk and maximize potential returns.
5. **Set clear goals and strategies**: Define your investment goals, risk tolerance, and investment horizon. Develop a strategy 